# Create dataframe needed for classification of event ratings,<br>using a uniform span between event dates
Local files needed:<br>
Events_FinalRatingsList.csv<br>
CleanLexisNexis.csv

In [153]:
import csv
import pandas as pd
from datetime import datetime as dt

## Read in the event dates and ratings

In [154]:
# Read in final table of event ratings. We can ignore FilterWords
eventstable = pd.read_csv('Events_FinalRatingsList.csv')
eventstable.head()

,EventID,Date,EventText,FilterWords,FinalRating
0,1001,01-31-2011,Syrian President Bashar al-Assad criticizes Is...,"extern, israel",2.0
1,1010,03-11-2011,Reuters reports ‘a large shipment of weapons ...,"omin, iraq",3.0
2,1011,03-12-2011,thousands of Syrian Kurds protested in Qamishl...,"all, kurd",-1.0
3,1023,03-26-2011,"Activists and human rights groups said, accord...","accord, kurd",0.0
4,1025,03-29-2011,"after a government appeal on 28 March, today t...","agenc, israel",1.0


In [155]:
eventstable.FinalRating.describe()

count    548.000000
mean       1.344282
std        1.467877
min       -1.000000
25%        0.000000
50%        2.000000
75%        3.000000
max        3.000000
Name: FinalRating, dtype: float64

In [156]:
print(eventstable.groupby('FinalRating')['EventID'].nunique())

FinalRating
-1.000000    121
 0.000000     19
 1.000000    102
 1.333333      2
 1.666667      3
 2.000000    155
 3.000000    146
Name: EventID, dtype: int64


## Create dataframe of events that are X days apart

In [157]:
finaleventdf = pd.DataFrame(index=None, columns=['EventID','Date','EventText','FinalRating'])

yescounter = 0
nocounter = 0
datecutoff = 6 # ensure there are at least this many days between events
minrating = 0   # do not include events with rating -1. These events did not receive a final rating based on algorithm.
dspan = 1000   # for storing the minimum number of days between events

# beginning reference date for calculating prior date cutoff is Jan 1, 2010
refdate=dt.strptime("12-01-2010", "%m-%d-%Y") 

for index, row in eventstable.iterrows():
    # calculate how many days have passed since the reference date and the current event date
    converteddate = dt.strptime(row['Date'], "%m-%d-%Y")
    d = converteddate - refdate
    
    # if rating is acceptable and days passed is greater than the cutoff:
    #    write event to dataframe finaleventdf
    #    set refdate to the latest event date
    #    increase counter
    #    record a new minimum dspan if days between events is smaller
    if row['FinalRating'] >= minrating and d.days > datecutoff:
        finaleventdf = finaleventdf.append({'EventID': row['EventID'],
                                      'Date': row['Date'],
                                      'EventText': row['EventText'],
                                      'FinalRating': int(round(row['FinalRating']))},
                                      ignore_index=True)
        refdate = converteddate
        yescounter += 1
        if d.days < dspan:
            dspan = d.days
    # else this event is too soon since the last event. Update counter
    else:
        nocounter += 1

finaleventdf['FinalRating'] = finaleventdf['FinalRating'].astype(int)
finaleventdf['EventID'] = finaleventdf['EventID'].astype(int)

print "Events included:", yescounter
print "Events excluded:", nocounter
print "Minimum day span between events:", dspan

Events included: 159
Events excluded: 389
Minimum day span between events: 7


In [158]:
print(finaleventdf.groupby('FinalRating')['EventID'].nunique())

FinalRating
0     6
1    35
2    54
3    64
Name: EventID, dtype: int64


In [159]:
finaleventdf

,EventID,Date,EventText,FinalRating
0,1001,01-31-2011,Syrian President Bashar al-Assad criticizes Is...,2
1,1010,03-11-2011,Reuters reports ‘a large shipment of weapons ...,3
2,1023,03-26-2011,"Activists and human rights groups said, accord...",0
3,1032,04-07-2011,"Of the 300,000 Syrian Kurds in region Hasakah ...",1
4,1040,04-16-2011,"After the new Syrian cabinet was sworn in, Pre...",1
5,2009,05-09-2011,The European Union institutes arms embargo and...,2
6,2017,05-16-2011,The Syrian army's siege across the country con...,2
7,2023,05-26-2011,"On 26 May, Haaretz reported that protests had ...",2
8,2029,06-02-2011,Witnesses said Syrian forces destroyed a numbe...,2
9,2038,06-13-2011,"By 13 June, helicopter gunships and tanks brea...",3


## Read in LexisNexis article data

In [160]:
# Read in csv of article data. We need article title and text by date. 
# Could also try publication title and publication type as features...
articletable = pd.read_csv('CleanLexisNexis.csv')
articletable.head()

,publication,date,title,length,publicationtype,text,year,month,day
0,The Atlanta Journal-Constitution,2010-01-03,Five pressing questions to answer in 2010,747,Newspapers,Will President Barack Obama regain his momentu...,2010,1,3
1,BBC Monitoring Middle East - Political Supplie...,2010-01-04,"Saudi foreign minister says Israel ""spoiled ch...",2196,Transcript,Text of report by Saudi-owned leading pan-Arab...,2010,1,4
2,BBC Monitoring Middle East - Political Supplie...,2010-01-08,Highlights of Iran parliamentary session.,1123,Transcript,Excerpt from report on parliamentary proceedin...,2010,1,8
3,Right Vision News,2010-01-09,Jordan:Way out for Obama,852,Newspaper,"Pakistan, Jan. 09 -- These are the worst of ti...",2010,1,9
4,The Nation (AsiaNet),2010-01-10,Zardari seeks UK help for access to EU markets,478,Newspaper,President Asif Ali Zardari has said that Brita...,2010,1,10


## Create the table of X-day-spaced events and article titles and texts from the preceding X days
This is the table to use for machine learning

In [161]:
# Update date formats for doing subtractions
finaleventdf['Date'] = pd.to_datetime(finaleventdf['Date'])
articletable['date'] = pd.to_datetime(articletable['date'])

In [162]:
mldataframe = pd.DataFrame(index=None, columns=['FinalRating','Title','Text'])
daystring = str(datecutoff) + ' days'

# for each event date, create a mini dataframe of articles occuring within the cutoff period
for index, row in finaleventdf.iterrows():
    eventdate = row['Date']
    eventcutoffdate = eventdate - pd.Timedelta(daystring)
    articlerange = (articletable['date'] >= eventcutoffdate) & (articletable['date'] < eventdate)
    thisrange = articletable.loc[articlerange]

    # for each article in the mini dataframe, write a row to a machine learning table
    # with the article title, text, and target event rating
    for artindex, artrow in thisrange.iterrows():
        mldataframe = mldataframe.append({'FinalRating': row['FinalRating'],
                                          'Title': artrow['title'],
                                          'Text': artrow['text']},
                                          ignore_index=True)

mldataframe['FinalRating'] = mldataframe['FinalRating'].astype(int)


In [163]:
mldataframe.head()

,FinalRating,Title,Text
0,2,Iran's foreign minister meets Syrian president,Text of report in English by Iranian news chan...
1,2,Analysis: Obama's State of the Union and US fo...,He will likely mention foreign issues and is u...
2,2,Israel alert to spillover of violence in Lebanon,NORTHERN ISRAELI BORDER -- Security officials ...
3,2,Canada gauging whether new Lebanon government ...,cclark@globeandmail.com How much of a banned t...
4,2,Congress mulls aid cut if Hizbullah controls g...,WASHINGTON - Continuing US aid to Lebanon will...


In [164]:
with open('RatingsandText.csv', 'w') as f_new:
    mldataframe.to_csv(f_new, header=True, index=False)
    f_new.close()